# MWS Secondary Target Selection

## MWS_SPECIAL_GC_VERYBRIGHT_SV

Maintainers:
  - Ting Li (selection/catalog)
  - Andrew Cooper (secondary target code)

Membership probability is calculated by Vasiliev 2018 for 150 GC (based on the spatial distribution and proper motion) in https://arxiv.org/abs/1807.09775

In this catalog we merge all the stars with membership probability > 0.5 into one list (0.5 is an arbitrary pick and we can easily change to a higher or lower threshold).

In [1]:
!pwd
!ls ../raw

/global/projecta/projectdirs/desi/external/mws/sv/MWS_SPECIAL_GC_VERYBRIGHT_SV/1/code
gc_p0_5_gaia_4DESI_v0.fits


In [2]:
# Parameters

target_class   = 'MWS_SPECIAL_GC_VERYBRIGHT_SV'
input_filename = 'gc_p0_5_gaia_4DESI_v0.fits'

OVERRIDE       = True
REF_CAT        = '?'
REF_EPOCH      = 2015.5

SELECT_MAG     = 'phot_g_mean_mag'
BRIGHT_LIMIT   = 9.0
FAINT_LIMIT    = 16.0

colmap = {
    'ra':        'RA',
    'dec':       'DEC',
    'source_id': 'REF_ID'
}

In [ ]:
from   importlib import reload
import sys
PYPATH = '/project/projectdirs/desi/external/mws/py/'
if PYPATH not in sys.path: sys.path.append(PYPATH)
import mwsecondary.mwsecondary as mws
reload(mws)

import os
import sys
import numpy as np

targets = mws.SecondaryTargets(target_class,input_filename,ref_cat=REF_CAT,ref_epoch=REF_EPOCH)
targets.print_info()

mask = np.repeat(True,len(targets.data))
mask   &= targets.select_finite_mag_range(SELECT_MAG,faint_limit=FAINT_LIMIT,bright_limit=BRIGHT_LIMIT)
mask   &= targets.select_finite_range('dec',lower_limit=-30.0,verbose=True)
mask   &= targets.select_unique('source_id',verbose=True)

# Adjust to secondary target data model and write output
output = targets.get_output(mask,cols=colmap,write=True,override=OVERRIDE)
print(output[0:3])